In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)


/Users/phaniram/anaconda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Use the retry module or similar alternatives.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [2]:
def variable_summaries(var):
    """Attach a lot of summaries to a Tensor (for TensorBoard visualization)."""
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean', mean)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        tf.summary.scalar('stddev', stddev)
        tf.summary.scalar('max', tf.reduce_max(var))
        tf.summary.scalar('min', tf.reduce_min(var))
        tf.summary.histogram('histogram', var)

In [3]:
def init_weights(shape):
    init_random_dist = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(init_random_dist)


def init_bias(shape):
    init_bias_vals = tf.constant(0.1, shape=shape)
    return tf.Variable(init_bias_vals)


def conv2d(x, W, name = 'Conv_2d'):
    with tf.name_scope(name):
        return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')


def max_pool_2by2(x, name= 'max_pooling'):
    with tf.name_scope(name):
        return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                              strides=[1, 2, 2, 1], padding='SAME')


def convolutional_layer(input_x, shape, name= 'Conv_Layer'):
    with tf.name_scope(name):
        with tf.name_scope('weights'):
            W = init_weights(shape)
            variable_summaries(W)
        with tf.name_scope('bias'):
            b = init_bias([shape[3]])
            variable_summaries(b)
        return tf.nn.relu(conv2d(input_x, W) + b)


def normal_full_layer(input_layer, size, name='FC_Layer'):
    
    input_size = int(input_layer.get_shape()[1])
    with tf.name_scope(name):
        with tf.name_scope('weights'):
            W = init_weights([input_size, size])
            variable_summaries(W)
        with tf.name_scope('bias'):
            b = init_bias([size])
            variable_summaries(b)
    return tf.matmul(input_layer, W) + b


In [4]:
tf.reset_default_graph()

In [5]:
sess = tf.Session()
x = tf.placeholder(tf.float32,shape=[None,784], name= 'x')
y_true = tf.placeholder(tf.float32,shape=[None,10], name = 'y')

x_image = tf.reshape(x,[-1,28,28,1])


convo_1 = convolutional_layer(x_image,shape=[6,6,1,32], name = 'conv1')
convo_1_pooling = max_pool_2by2(convo_1)


convo_2 = convolutional_layer(convo_1_pooling,shape=[6,6,32,64], name = 'conv2')
convo_2_pooling = max_pool_2by2(convo_2)

convo_2_flat = tf.reshape(convo_2_pooling,[-1,7*7*64])
full_layer_one = tf.nn.relu(normal_full_layer(convo_2_flat,1024))

hold_prob = tf.placeholder(tf.float32, name = 'hold_prob')

with tf.name_scope('drop_out'):
    full_one_dropout = tf.nn.dropout(full_layer_one,keep_prob=hold_prob)
    tf.summary.scalar('drop_out_hold_probability', hold_prob)

y_pred = normal_full_layer(full_one_dropout,10)

with tf.name_scope('cross_entropy'):
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_true,logits=y_pred))
    tf.summary.scalar('cross_entropy_val', cross_entropy)
    
optimizer = tf.train.AdamOptimizer(learning_rate=0.0001)
train = optimizer.minimize(cross_entropy)

merged = tf.summary.merge_all()
train_writer = tf.summary.FileWriter('./train', sess.graph)

init = tf.global_variables_initializer()

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [6]:
steps = 5000    
sess.run(init)

for i in range(steps):

    batch_x , batch_y = mnist.train.next_batch(50)

    summary, loss = sess.run([merged, train],feed_dict={x:batch_x,y_true:batch_y,hold_prob:0.5})
    #tf.summary.scalar('loss', loss)
    train_writer.add_summary(summary, i)

    # PRINT OUT A MESSAGE EVERY 100 STEPS
    if i%100 == 0:

        print('Currently on step {}'.format(i))
        print('Accuracy is:')
        # Test the Train Model
        matches = tf.equal(tf.argmax(y_pred,1),tf.argmax(y_true,1))

        acc = tf.reduce_mean(tf.cast(matches,tf.float32))

        print(sess.run(acc,feed_dict={x:mnist.test.images,y_true:mnist.test.labels,hold_prob:1.0}))
        print('\n')

Currently on step 0
Accuracy is:
0.1446


Currently on step 100
Accuracy is:
0.8604


Currently on step 200
Accuracy is:
0.9131


Currently on step 300
Accuracy is:
0.9248


Currently on step 400
Accuracy is:
0.9381


Currently on step 500
Accuracy is:
0.9437


Currently on step 600
Accuracy is:
0.954




KeyboardInterrupt: 

In [7]:
(2,)+(3,5)

(2, 3, 5)